In [6]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm

# --- 常數定義 ---
DAYS_PER_YEAR = 365.25
SECONDS_PER_DAY = 24 * 3600

# --- Gate.io K 線資料抓取 ---
def get_gateio_kline(currency_pair: str, interval: str = "1h", limit: int = 720) -> pd.DataFrame:
    """
    從 Gate.io API 取得歷史 K 線資料
    文件: https://www.gate.io/docs/developers/apiv4/en/#market-k-line-chart
    """
    base_url = "https://api.gateio.ws/api/v4/spot/candlesticks"
    params = {
        "currency_pair": currency_pair.upper(),
        "interval": interval,
        "limit": limit
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()

    # API 回傳格式: [[timestamp, volume_quote, close, high, low, open, volume_base, closed], ...]
    df = pd.DataFrame(data, columns=[
        "timestamp", "volume_quote", "close", "high", "low", "open", "volume_base", "closed"
    ])

    df["timestamp"] = pd.to_datetime(df["timestamp"].astype(float), unit="s", utc=True)
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)

    # 依時間排序（API 回傳通常是最新在前）
    df = df.sort_values("timestamp").reset_index(drop=True)
    return df[["timestamp", "open", "high", "low", "close"]]


def get_bybit_kline(currency_pair: str, interval: str = "1h", limit: int = 720) -> pd.DataFrame:
    """
    從 Bybit API 取得歷史 K 線資料
    文件: https://bybit-exchange.github.io/docs/v5/market/kline
    """
    # 轉換 interval 格式：Gate.io 的 "1h" -> Bybit 的 "60"
    interval_map = {
        "1m": "1", "3m": "3", "5m": "5", "15m": "15", "30m": "30",
        "1h": "60", "2h": "120", "4h": "240", "6h": "360", "12h": "720",
        "1d": "D", "1w": "W", "1M": "M"
    }
    bybit_interval = interval_map.get(interval, interval)
    
    base_url = "https://api.bybit.com/v5/market/kline"
    params = {
        "category": "spot",
        "symbol": currency_pair.upper().replace("_", ""),  # BTC_USDT -> BTCUSDT
        "interval": bybit_interval,
        "limit": limit
    }
    
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # 檢查 API 回傳狀態
    if data.get("retCode") != 0:
        raise Exception(f"Bybit API Error: {data.get('retMsg')}")
    
    # API 回傳格式: result.list = [[timestamp, open, high, low, close, volume, turnover], ...]
    klines = data["result"]["list"]
    df = pd.DataFrame(klines, columns=[
        "timestamp", "open", "high", "low", "close", "volume", "turnover"
    ])
    
    df["timestamp"] = pd.to_datetime(df["timestamp"].astype(float), unit="ms", utc=True)
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
    
    # Bybit 回傳的是最新在前，需要反轉
    df = df.sort_values("timestamp").reset_index(drop=True)
    return df[["timestamp", "open", "high", "low", "close"]]

In [13]:
# --- 核心策略計算 ---
def calculate_optimal_market_making_params(
    asset: str,
    mid_price: float,
    daily_volatility_pct: float,
    target_order_fill_prob: float,
    order_refresh_time_sec: int,
    stop_loss_risk_prob: float,
    max_holding_time_days: float = 30.0,
    profit_factor: float = 2.0,
    print_result: bool = False
    ) -> dict:
    """
    根據 GBM 波動率模型，計算最優造市參數
    """
    daily_volatility = daily_volatility_pct / 100.0
    annual_volatility = daily_volatility * np.sqrt(DAYS_PER_YEAR)
    dt_order = order_refresh_time_sec / (DAYS_PER_YEAR * SECONDS_PER_DAY)
    dt_loss = max_holding_time_days / DAYS_PER_YEAR

    # 基礎掛單價差
    p_half_order = target_order_fill_prob / 2.0
    Z_order = norm.ppf(p_half_order)
    base_spread_pct = (annual_volatility * np.sqrt(dt_order) * np.abs(Z_order)) * 100

    # 止盈與止損
    profit_taking_spread_pct = base_spread_pct * profit_factor
    p_half_loss = stop_loss_risk_prob / 2.0
    Z_loss = norm.ppf(p_half_loss)
    stop_loss_spread_pct = (annual_volatility * np.sqrt(dt_loss) * np.abs(Z_loss)) * 100

    return {
        "asset": asset,
        "current_mid_price": mid_price,
        "order_refresh_time_sec": int(order_refresh_time_sec),
        "bid_spread": round(base_spread_pct, 4),
        "ask_spread": round(base_spread_pct, 4),
        "long_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "short_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "stop_loss_spread": round(stop_loss_spread_pct, 4),
        "Z_score_order": round(np.abs(Z_order), 4),
        "Z_score_stop_loss": round(np.abs(Z_loss), 4)
    }


# --- 主流程：自動從交易所取得波動率並計算策略 ---
def calculate_from_exchanges(
                          exchange_name: str = "gateio",
                          currency_pair: str = "XRP_USDT", 
                          interval: str = "1h",
                          target_order_fill_prob: float = 0.50,
                          order_refresh_time_sec: int = 60,
                          stop_loss_risk_prob: float = 0.5,
                          profit_factor: float = 1, 
                          print_result: bool = False
                          ):
    """
    從交易所取得歷史資料，自動估算波動率並計算造市策略參數
    支援交易所: gateio, bybit
    """
    function_map = {
        "gateio": get_gateio_kline,
        "bybit": get_bybit_kline
    }
    
    if exchange_name not in function_map:
        raise ValueError(f"不支援的交易所: {exchange_name}. 支援: {list(function_map.keys())}")

    df = function_map[exchange_name](currency_pair, interval=interval, limit=720)

    # 計算對數報酬率
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))
    hourly_vol = df["log_return"].std()

    # 換算成日化波動率
    daily_vol = hourly_vol * np.sqrt(24)
    print(f"📊 {currency_pair} 日化波動率估計值: {daily_vol*100:.2f}%")

    params = {
        "asset": currency_pair,
        "mid_price": df["close"].iloc[-1],
        "daily_volatility_pct": daily_vol * 100,
        "target_order_fill_prob": target_order_fill_prob,
        "order_refresh_time_sec": order_refresh_time_sec,
        "stop_loss_risk_prob": stop_loss_risk_prob,
        "max_holding_time_days": 1,
        "profit_factor": profit_factor
    }

    result = calculate_optimal_market_making_params(**params)

    if print_result:
        print("\n--- 🔬 最優造市參數計算結果 ---")
        print(f"--- 使用 {exchange_name.upper()} 交易所 ---")
        for key, value in result.items():
            if "_spread" in key or "time_sec" in key:
                unit = "%" if "spread" in key else "秒"
                print(f"{key:<30}: {value:>.4f} {unit}")
            elif key == "current_mid_price":
                print(f"{key:<30}: {value:>.4f} USDT")
            elif key == "asset":
                print(f"{key:<30}: {value}")

    return result

In [14]:
calculate_from_exchanges("bybit", "MNT_USDT", interval="1h", profit_factor=3, print_result=True)

📊 MNT_USDT 日化波動率估計值: 5.20%

--- 🔬 最優造市參數計算結果 ---
--- 使用 BYBIT 交易所 ---
asset                         : MNT_USDT
current_mid_price             : 1.0717 USDT
order_refresh_time_sec        : 60.0000 秒
bid_spread                    : 0.0925 %
ask_spread                    : 0.0925 %
long_profit_taking_spread     : 0.2774 %
short_profit_taking_spread    : 0.2774 %
stop_loss_spread              : 3.5094 %


{'asset': 'MNT_USDT',
 'current_mid_price': 1.0717,
 'order_refresh_time_sec': 60,
 'bid_spread': 0.0925,
 'ask_spread': 0.0925,
 'long_profit_taking_spread': 0.2774,
 'short_profit_taking_spread': 0.2774,
 'stop_loss_spread': 3.5094,
 'Z_score_order': 0.6745,
 'Z_score_stop_loss': 0.6745}